### Complete Testrun of Logos Service
This notebook provides a complete testrun of logos. It has own models, classifies them under real conditions and sends requests to the k >= 1 most suitable models.

In [ ]:
from logos.classification.classification_manager import ClassificationManager
from logos.scheduling.scheduling_fcfs import FCFSScheduler
from logos.scheduling.scheduling_manager import SchedulingManager
from pprint import pprint

In [ ]:
policy = {
    "id": 0,
    "name": "lax_all",
    "entity_id": 0,
    "description": "Somehow all LLMs that come into mind",
    "threshold_privacy": "CLOUD_NOT_IN_EU_BY_US_PROVIDER",
    "threshold_latency": 32,
    "threshold_accuracy": 32,
    "threshold_cost": -32,
    "threshold_quality": 32,
    "priority": 255,
    "topic": "",
}

In [ ]:
from test_model_data import prepare_model_data
models = prepare_model_data()
classifier = ClassificationManager(models)
sm = SchedulingManager(FCFSScheduler())
sm.run()

In [ ]:
prompt = "Tell me a short joke"

In [ ]:
results = classifier.classify(prompt, policy)

In [ ]:
tid = sm.add_request(dict(), results)

# Wait for this task to be executed
while not sm.is_finished(tid):
    pass

out = sm.get_result()

In [ ]:
print(out.models)

In [ ]:
with open("key.txt", "r") as f:
    key = f.readline().strip().replace("\t", "").replace("\n", "")

In [ ]:
from test_model_data import get_from_id, test_send_to_azure, create_html, test_send_to_webui

data = dict()

for tpl in out.models:
    model = get_from_id(models, tpl[0])
    if tpl[0] in (0, 1):
        response = test_send_to_azure(model, prompt, key if tpl[0] == 0 else "")
        print(f"Model {model['name']} sent to Azure")
    else:
        response = test_send_to_webui(model, prompt, key if tpl[0] == 0 else "")
        print(f"Model {model['name']} sent to Openwebui")
    try:
        data[model['name']] = response["choices"][0]["message"]["content"]
        # pprint(response["choices"][0]["message"]["content"])
    except:
        pprint(response)
    # print()
    # print()
    

In [ ]:
data

In [ ]:

create_html(data)

In [ ]:
sm.stop()

In [ ]:
from test_model_data import test_send_to_webui
openwebui = ""

In [ ]:
response = test_send_to_webui("", prompt, openwebui)
print(f"Model xxx sent to Webui:")
try:
    pprint(response)
except:
    pprint(response)

In [ ]:
import requests


def chat_with_model(token):
    url = 'https://gpu.aet.cit.tum.de/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    data = {
      "model": "deepseek-r1:70b",
      "messages": [
        {
          "role": "user",
          "content": "Tell me a short joke"
        }
      ]
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()

pprint(chat_with_model(openwebui))